In [7]:
!pip install langchain_core

In [8]:
!pip install llama-cpp-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 MB 19.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
Y
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 3.1 MB/s eta 0:00:00
Y
y
Y
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.3.16-cp312-cp312-linux_x86_64.whl size=4422309 sha256=cb84023261dcdf19990805f326d610cad3f559ad00a3f5705ef10cdf8a6191c8
  Stored in directory: /root/.cache/pip/wheels/90/82/ab/8784ee3fb99ddb07fd36a679ddbe63122cc07718f6c1eb3be8
Successfully built llama-cpp-python


In [15]:
import langchain, langchain_community, langchain_core, langchain_classic

In [3]:
from langchain_community.llms import LlamaCpp
from huggingface_hub import hf_hub_download

# Make sure the model path is correct for your system!
# Download the model from Hugging Face
model_name = "microsoft/Phi-3-mini-4k-instruct-gguf"
model_file = "Phi-3-mini-4k-instruct-q4.gguf"
model_path = hf_hub_download(repo_id=model_name, filename=model_file)

llm = LlamaCpp(
    model_path=model_path,
    n_gpu_layers=-1,
    max_tokens=500,
    n_ctx=2048,
    seed=42,
    verbose=False
)

llama_context: n_batch is less than GGML_KQ_MASK_PAD - increasing to 64
llama_context: n_ctx_per_seq (2048) < n_ctx_train (4096) -- the full capacity of the model will not be utilized


In [4]:
llm.invoke("Hi! My name is Maarten. What is 1 + 1?")

''

In [9]:
import langchain_community
from langchain_core.prompts import PromptTemplate

# Create a prompt template with the "input_prompt" variable
template = """<s><|user|>
{input_prompt}
<|end|>
<|assistant|>"""
prompt = PromptTemplate(
template=template,
input_variables=["input_prompt"]
)

In [10]:
basic_chain = prompt | llm

In [11]:
# Use the chain
basic_chain.invoke(
{
"input_prompt": "Hi! My name is Maarten. What is 1 + 1?",
}
)

/usr/local/lib/python3.12/dist-packages/llama_cpp/llama.py:1242: RuntimeWarning: Detected duplicate leading "<s>" in prompt, this will likely reduce response quality, consider removing it...
  warnings.warn(


" Hello Maarten! The answer to 1 + 1 is 2. It's a basic arithmetic addition problem where you combine one unit with another, resulting in two units in total."

In [17]:
from langchain_classic.chains import LLMChain
# Create a chain for the title of our story
template = """<s><|user|>
Create a title for a story about
{summary}
. Only return the title.
<|end|>
<|assistant|>"""
title_prompt = PromptTemplate(template=template, input_variables=["summary"])
title = LLMChain(llm=llm, prompt=title_prompt, output_key="title")


/tmp/ipython-input-3287247481.py:10: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use `RunnableSequence, e.g., `prompt | llm`` instead.
  title = LLMChain(llm=llm, prompt=title_prompt, output_key="title")


In [18]:
title.invoke({"summary": "a girl that lost her mother"})

/usr/local/lib/python3.12/dist-packages/llama_cpp/llama.py:1242: RuntimeWarning: Detected duplicate leading "<s>" in prompt, this will likely reduce response quality, consider removing it...
  warnings.warn(


{'summary': 'a girl that lost her mother',
 'title': ' "Whispers of a Mother\'s Love: A Journey Through Grief"'}

In [20]:
# Create a chain for the character description using the summary and title
template = """<s><|user|>
Describe the main character of a story about
{summary}
 with the
title
{title}
. Use only two sentences.<|end|>
<|assistant|>"""
character_prompt = PromptTemplate(
template=template, input_variables=["summary", "title"]
)
character = LLMChain(llm=llm, prompt=character_prompt,
output_key="character")


In [21]:
# Create a chain for the story using the summary, title, and character description
template = """<s><|user|>
Create a story about
{summary}
 with the title
{title}
. The main
character is:
{character}
. Only return the story and it cannot be
longer than one paragraph. <|end|>
<|assistant|>"""
story_prompt = PromptTemplate(
template=template, input_variables=["summary", "title", "character"]
)
story = LLMChain(llm=llm, prompt=story_prompt,
output_key="story")

In [22]:
# Combine all three components to create the full chain
llm_chain = title | character | story

In [23]:
llm_chain.invoke("a girl that lost her mother")

{'summary': 'a girl that lost her mother',
 'title': ' "A Mother\'s Absence: Navigating Life Without a Guiding Light"',
 'character': " The protagonist, Amelia, is a resilient and compassionate teenage girl grappling with the overwhelming void left by her mother's sudden passing. As she navigates life without her guiding light, Amelia discovers inner strength, forges new connections, and learns to find solace in cherished memories of her beloved mother.",
 'story': " Title: A Mother's Absence: Navigating Life Without a Guiding Light\n\nAmelia, a resilient and compassionate teenage girl, struggled to come to terms with the sudden loss of her mother. With each passing day, she felt an overwhelming void that seemed impossible to fill. But Amelia's determination to honor her mother's memory pushed her forward in a world without her guiding light. She discovered inner strength and learned to forge new connections as friends and mentors stepped into the role her mother once held. Through che

In [24]:
# Let's give the LLM our name
basic_chain.invoke({"input_prompt": "Hi! My name is Maarten. What is 1 + 1?"})

" Hello Maarten! The answer to 1 + 1 is 2. It's a basic arithmetic addition where one unit added to another unit equals two units in total."

In [25]:
# Next, we ask the LLM to reproduce the name
basic_chain.invoke({"input_prompt": "What is my name?"})

" I'm sorry, but as an AI, I don't have the ability to know personal information about individuals unless it has been shared with me in the course of our conversation. If you're looking for help identifying a name related to your context or needs, feel free to provide more details!"

In [26]:
# Create an updated prompt template to include a chat history
template = """<s><|user|>Current conversation:
{chat_history}
{input_prompt}
<|end|>
<|assistant|>"""
prompt = PromptTemplate(
template=template,
input_variables=["input_prompt", "chat_history"]
)


In [29]:
from langchain_classic.memory import ConversationBufferMemory

# Define the type of memory we will use
memory = ConversationBufferMemory(memory_key="chat_history")

# Chain the LLM, prompt, and memory together
llm_chain = LLMChain(
prompt=prompt,
llm=llm,
memory=memory
)

/tmp/ipython-input-1786352283.py:4: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key="chat_history")


In [30]:
# Generate a conversation and ask a basic question
llm_chain.invoke({"input_prompt": "Hi! My name is Maarten. What is 1 + 1?"})

/usr/local/lib/python3.12/dist-packages/llama_cpp/llama.py:1242: RuntimeWarning: Detected duplicate leading "<s>" in prompt, this will likely reduce response quality, consider removing it...
  warnings.warn(


{'input_prompt': 'Hi! My name is Maarten. What is 1 + 1?',
 'chat_history': '',
 'text': " The answer to 1 + 1 is 2. Hello, Maarten! It's a simple arithmetic question where when you add one unit to another unit, you get two units in total."}

In [31]:
# Does the LLM remember the name we gave it?
llm_chain.invoke({"input_prompt": "What is my name?"})

{'input_prompt': 'What is my name?',
 'chat_history': "Human: Hi! My name is Maarten. What is 1 + 1?\nAI:  The answer to 1 + 1 is 2. Hello, Maarten! It's a simple arithmetic question where when you add one unit to another unit, you get two units in total.",
 'text': " Your name is Assistant. Nice to meet you, Maarten! I'm here to help with any questions or tasks you need assistance with."}

In [32]:
llm_chain.invoke({"input_prompt": "What did the conversation we have till now?"})

{'input_prompt': 'What did the conversation we have till now?',
 'chat_history': "Human: Hi! My name is Maarten. What is 1 + 1?\nAI:  The answer to 1 + 1 is 2. Hello, Maarten! It's a simple arithmetic question where when you add one unit to another unit, you get two units in total.\nHuman: What is my name?\nAI:  Your name is Assistant. Nice to meet you, Maarten! I'm here to help with any questions or tasks you need assistance with.",
 'text': ' In the conversation, Maarten asked for the result of a simple arithmetic question (1 + 1), to which the AI replied that it equals 2. Then, Maarten inquired about his name, prompting the AI to mistakenly refer to itself as "Assistant" instead of acknowledging Maarten\'s actual name. The AI introduced itself and offered assistance to Maarten afterward.'}

In [33]:
from langchain_classic.memory import ConversationBufferWindowMemory
# Retain only the last 2 conversations in memory
memory = ConversationBufferWindowMemory(k=2,
memory_key="chat_history")
# Chain the LLM, prompt, and memory together
llm_chain = LLMChain(
prompt=prompt,
llm=llm,
memory=memory
)

/tmp/ipython-input-1476245510.py:3: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferWindowMemory(k=2,


In [34]:
# Ask two questions and generate two conversations in its memory
llm_chain.predict(input_prompt="Hi! My name is Maarten and I am 33 years old. What is 1 + 1?")
llm_chain.predict(input_prompt="What is 3 + 3?")

/usr/local/lib/python3.12/dist-packages/llama_cpp/llama.py:1242: RuntimeWarning: Detected duplicate leading "<s>" in prompt, this will likely reduce response quality, consider removing it...
  warnings.warn(


' Hello Maarten! 3 + 3 equals 6. If you have any other questions or need further assistance, feel free to ask!'

In [35]:
# Check whether it knows the name we gave it
llm_chain.invoke({"input_prompt":"What is my name?"})

{'input_prompt': 'What is my name?',
 'chat_history': "Human: Hi! My name is Maarten and I am 33 years old. What is 1 + 1?\nAI:  Hello Maarten! It's a pleasure to meet you. 1 + 1 equals 2.\n\nIf you have any other questions or need assistance with something, feel free to ask!\nHuman: What is 3 + 3?\nAI:  Hello Maarten! 3 + 3 equals 6. If you have any other questions or need further assistance, feel free to ask!",
 'text': ' Your name is Maarten. Nice to have you here!'}

In [36]:
# Check whether it knows the age we gave it
llm_chain.invoke({"input_prompt":"What is my age?"})

{'input_prompt': 'What is my age?',
 'chat_history': 'Human: What is 3 + 3?\nAI:  Hello Maarten! 3 + 3 equals 6. If you have any other questions or need further assistance, feel free to ask!\nHuman: What is my name?\nAI:  Your name is Maarten. Nice to have you here!',
 'text': " As an AI, I don't have access to personal information such as your age. However, you can determine your age by subtracting the year you were born from the current year. If privacy concerns are a factor, remember that it's best not to share sensitive personal details online for security reasons."}

In [37]:
# Create a summary prompt template
summary_prompt_template = """<s><|user|>Summarize the
conversations and update with the new lines.
Current summary:
{summary}
new lines of conversation:
{new_lines}
New summary:<|end|>
<|assistant|>"""
summary_prompt = PromptTemplate(
input_variables=["new_lines", "summary"],
template=summary_prompt_template
)


In [38]:
from langchain_classic.memory import ConversationSummaryMemory
# Define the type of memory we will use
memory = ConversationSummaryMemory(
llm=llm,
memory_key="chat_history",
prompt=summary_prompt
)
# Chain the LLM, prompt, and memory together
llm_chain = LLMChain(
prompt=prompt,
llm=llm,
memory=memory
)

/tmp/ipython-input-592517272.py:3: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationSummaryMemory(


In [39]:
llm_chain.invoke({"input_prompt": "Hi! My name is Maarten. What is 1 + 1?"})
llm_chain.invoke({"input_prompt": "What is my name?"})

/usr/local/lib/python3.12/dist-packages/llama_cpp/llama.py:1242: RuntimeWarning: Detected duplicate leading "<s>" in prompt, this will likely reduce response quality, consider removing it...
  warnings.warn(


{'input_prompt': 'What is my name?',
 'chat_history': ' Maarten introduces himself and inquires about the sum of 1 + 1, to which the AI confirms that it equals 2, explaining the concept of basic arithmetic addition.',
 'text': " Based on the information provided in the current conversation, your name is Maarten. However, since this seems to be a hypothetical scenario and you haven't directly mentioned it yourself, I can only infer that from the dialogue presented. If not for context clues, there would be no way to know your actual name as per our interaction guidelines here."}

In [40]:
# Check whether it has summarized everything thus far
llm_chain.invoke({"input_prompt": "What was the first question I asked?"})

{'input_prompt': 'What was the first question I asked?',
 'chat_history': " Maarten introduces himself and asks about the sum of 1 + 1, to which the AI confirms it equals 2, explaining basic arithmetic addition. Additionally, based on the conversation context, the AI infers that the human's name is likely Maarten but cannot confirm without direct information from the user.",
 'text': ' The first question you asked was, "to which the AI confirms it equals 2, explaining basic arithmetic addition." However, if we\'re strictly looking for an initial inquiry about Maarten introducing himself and asking a related question, then the more appropriate interpretation would be: "Maarten introduces himself and asks what the sum of 1 + 1 is."'}

In [41]:
# Check what the summary is thus far
memory.load_memory_variables({})

{'chat_history': " Maarten introduces himself and asks about the sum of 1 + 1, to which the AI confirms it equals 2, explaining basic arithmetic addition. The AI also identifies that while suggesting the human's name as likely Maarten based on context, direct user confirmation is required for certainty. Additionally, when asked about the first question, the AI reiterates the inquiry about the sum of 1 + 1 and clarifies it as Maarten's initial interaction."}